In [1]:
import json
import pandas as pd
from collections import Counter

from nltk import word_tokenize, sent_tokenize

from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', 200)

In [2]:
with open("../data/href2synopsis.json", encoding="utf-8") as f:
    href2synopsis = json.load(f)

# BOW stats

In [15]:
data = pd.DataFrame()

data["href"] = href2synopsis.keys()
data["synopsis"] = href2synopsis.values()

In [16]:
data["char_total"] = data["synopsis"].progress_apply(len)
data["token_total"] = data["synopsis"].progress_apply(lambda x: len(word_tokenize(x)))
data["sent_total"] = data["synopsis"].progress_apply(lambda x: len(sent_tokenize(x)))

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

In [17]:
data.describe().astype(int)

,char_total,token_total,sent_total
count,19393,19393,19393
mean,493,99,4
std,359,72,3
min,0,0,0
25%,233,46,2
50%,433,87,4
75%,683,138,7
max,5804,1201,75


In [18]:
stats = data.drop(columns=["href", "synopsis"])

# Outlier detection

[The svm.OneClassSVM is known to be sensitive to outliers and thus does not perform very well for outlier detection](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection)

In [19]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [20]:
clf = LocalOutlierFactor(n_neighbors=20)
preds = clf.fit_predict(stats)
# clf.negative_outlier_factor_
data["pred"] = preds

In [12]:
# data[data["pred"] == -1]["synopsis"].sample(10).to_list()

In [22]:
data[data["pred"] == -1].sort_values(by="sent_total").head()

,href,synopsis,char_total,token_total,sent_total,pred
9710,/11107-doctor-x-3,Freelance surgeon Daimon Michiko gets embroiled in an escalating struggle for supremacy between the factions of Totei University Hospital and Saikyo University Hospital which represent Japan’s eas...,302,47,1,-1
9261,/741125-the-farewell-song,"~~ Adapted from the novel ""Li Ge"" (离歌) by Rao Xue Man (饶雪漫).",60,20,1,-1
9251,/741959-qian-duo-tao-hua-yi-shi-kai,"~~ Adapted from the web novel ""Qian Duo Tao Hua Yi Shi Kai"" (千朵桃花一世开) by Sui Yu Er An (随宇而安).",93,27,1,-1
18883,/689819-hidamari-no-basho-hatsukoi,Mobile drama.,13,3,1,-1
8963,/702939-shuang-xi,"~~ Adapted from the novel ""Shuang Xi"" (双喜) by Lang Lang (郎朗).",61,19,1,-1
